In [4]:
import csv
import json
import re # Per la conversione dei tipi

def convert_value(value):
    """
    Tenta di convertire un valore da stringa al tipo Python appropriato (int, float, bool, None).
    Altrimenti, lo restituisce come stringa.
    """
    value = value.strip()
    if not value:
        return None # Tratta stringhe vuote come None (null in JSON)

    # Tentativo di conversione a booleano
    if value.lower() == 'true':
        return True
    if value.lower() == 'false':
        return False
    if value.lower() == 'null':
        return None # Se il CSV contiene "null" esplicito

    # Tentativo di conversione a numero intero o decimale
    try:
        return int(value)
    except ValueError:
        try:
            return float(value)
        except ValueError:
            pass
    
    # Se non è nessuno dei precedenti, restituisci la stringa
    return value

def csv_to_jsonl(input_csv_file, output_jsonl_file, delimiter=','):
    """
    Converte un file CSV in un file JSONL.
    
    Args:
        input_csv_file (str): Percorso del file CSV di input.
        output_jsonl_file (str): Percorso del file JSONL di output.
        delimiter (str): Il delimitatore usato nel file CSV (es. ',', ';').
    """
    try:
        with open(input_csv_file, mode='r', newline='', encoding='utf-8') as infile, \
             open(output_jsonl_file, mode='w', encoding='utf-8') as outfile:
            
            reader = csv.reader(infile, delimiter=delimiter)
            
            # Leggi l'intestazione (header)
            header = [h.strip() for h in next(reader)]
            
            print(f"Intestazioni CSV rilevate: {header}")
            
            # Processa ogni riga di dati
            for i, row in enumerate(reader):
                if len(row) != len(header):
                    print(f"Attenzione: La riga {i+2} ha un numero di colonne ({len(row)}) diverso dall'intestazione ({len(header)}). Verrà saltata o trattata parzialmente.")
                    # Puoi scegliere di saltare la riga, loggare l'errore o gestirla diversamente
                    continue 
                
                record = {}
                for j, field_name in enumerate(header):
                    # Pulisci il nome del campo da spazi extra, caratteri non validi per chiavi JSON (raro ma possibile)
                    # e convertilo in camelCase o snake_case se preferisci.
                    # Per semplicità, qui usiamo il nome esatto dell'intestazione.
                    
                    # Converti il valore al tipo corretto
                    record[field_name] = convert_value(row[j])
                
                # Scrivi l'oggetto JSON su una nuova riga
                outfile.write(json.dumps(record, ensure_ascii=False) + '\n')
        
        print(f"Conversione completata: '{input_csv_file}' è stato convertito in '{output_jsonl_file}'")

    except FileNotFoundError:
        print(f"Errore: File non trovato '{input_csv_file}'")
    except Exception as e:
        print(f"Si è verificato un errore durante la conversione: {e}")



In [5]:
import csv
import json
import re # Per la conversione dei tipi
import os



# Convert all the CSV files in a input_folder into output_folder as JSONL files
def convert_all_csv_to_jsonl(input_folder, output_folder, delimiter=','):
    """
    Converte tutti i file CSV in una cartella in file JSONL nella cartella di output.
    
    Args:
        input_folder (str): Cartella contenente i file CSV di input.
        output_folder (str): Cartella dove salvare i file JSONL di output.
        delimiter (str): Il delimitatore usato nei file CSV (es. ',', ';').
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            input_file = os.path.join(input_folder, filename)
            output_file = os.path.join(output_folder, filename.replace('.csv', '.jsonl'))
            csv_to_jsonl(input_file, output_file, delimiter)





In [6]:
convert_all_csv_to_jsonl('scores_gemini', 'outputs_folder/scores_gemini', delimiter=',')

Intestazioni CSV rilevate: ['Sentence', 'Translation', 'Score']
Conversione completata: 'scores_gemini\scored_cerbero_base.csv' è stato convertito in 'outputs_folder/scores_gemini\scored_cerbero_base.jsonl'
Intestazioni CSV rilevate: ['Sentence', 'Translation', 'Score']
Conversione completata: 'scores_gemini\scored_cerbero_detailed.csv' è stato convertito in 'outputs_folder/scores_gemini\scored_cerbero_detailed.jsonl'
Intestazioni CSV rilevate: ['Sentence', 'Translation', 'Score']
Conversione completata: 'scores_gemini\scored_cerbero_few_shot.csv' è stato convertito in 'outputs_folder/scores_gemini\scored_cerbero_few_shot.jsonl'
Intestazioni CSV rilevate: ['Sentence', 'Translation', 'Score']
Conversione completata: 'scores_gemini\scored_cerbero_role-based.csv' è stato convertito in 'outputs_folder/scores_gemini\scored_cerbero_role-based.jsonl'
Intestazioni CSV rilevate: ['Sentence', 'Translation', 'Score']
Conversione completata: 'scores_gemini\scored_cerbero_teacher_student.csv' è sta

In [ ]:
convert_all_csv_to_jsonl('scores_prometheus', 'outputs_folder/scores_prometheus', delimiter=',')
convert_all_csv_to_jsonl('scores_manual', 'outputs_folder/scores_manual', delimiter=';')
convert_all_csv_to_jsonl('translations', 'outputs_folder/translations', delimiter=',')
# TODO when ready convert also the gemini scores

csv_to_jsonl('report.csv', 'outputs_folder/report.jsonl', delimiter=',')

Intestazioni CSV rilevate: ['Author', 'Date', 'Region', 'Sentence', 'translation', 'Score']
Conversione completata: 'scores_prometheus\scored_cerbero_base.csv' è stato convertito in 'outputs_folder/scores_prometheus\scored_cerbero_base.jsonl'
Intestazioni CSV rilevate: ['Author', 'Date', 'Region', 'Sentence', 'translation', 'Score']
Conversione completata: 'scores_prometheus\scored_cerbero_detailed.csv' è stato convertito in 'outputs_folder/scores_prometheus\scored_cerbero_detailed.jsonl'
Intestazioni CSV rilevate: ['Author', 'Date', 'Region', 'Sentence', 'translation', 'Score']
Conversione completata: 'scores_prometheus\scored_cerbero_few_shot.csv' è stato convertito in 'outputs_folder/scores_prometheus\scored_cerbero_few_shot.jsonl'
Intestazioni CSV rilevate: ['Author', 'Date', 'Region', 'Sentence', 'translation', 'Score']
Conversione completata: 'scores_prometheus\scored_cerbero_role-based.csv' è stato convertito in 'outputs_folder/scores_prometheus\scored_cerbero_role-based.jsonl'


In [ ]:
# for each file csv in the folder, rename the file from judge_models-gemini-1.5-flash-latest_cerbero_base.csv to scored_cerbero_base.csv where the formatting is scored_model_prompt.csv

llm_eval_path = "scores_gemini/"

for filename in os.listdir(llm_eval_path):
    if filename.endswith('.csv'):
        new_filename = filename.replace('judge_models-', 'scored_').replace('_gemini-1.5-flash-latest', '').replace('.csv', '.csv')
        old_path = os.path.join(llm_eval_path, filename)
        new_path = os.path.join(llm_eval_path, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed {filename} to {new_filename}")

In [ ]:
# Final_scores

